In [5]:
from diffusers import DDIMScheduler
from diffusers import StableDiffusionPipeline, AutoencoderKL
import torch

RuntimeError: Failed to import diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion because of the following error (look up to see its traceback):
No module named 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion'

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
from multi_token_clip import MultiTokenCLIPTokenizer
from diffusers import DiffusionPipeline

# 将 model_path 设置为 Stable-Diffusion-v1.5 的模型路径或名字
model_or_name = "../SDXL"


image_encoder_path = "../ip-composition-adapter/sdxl_models/image_encoder"

ip_ckpt = "../ip-composition-adapter/sdxl_models/ip-adapter_sdxl.bin"

scheduler = DDIMScheduler.from_pretrained(model_or_name, subfolder="scheduler")
tokenizer = MultiTokenCLIPTokenizer.from_pretrained(model_or_name, subfolder="tokenizer")
vae = AutoencoderKL.from_pretrained(model_or_name, subfolder="vae").to(dtype=torch.float32)
# pipeline = StableDiffusionPipeline.from_pretrained(
#     model_or_name, scheduler=scheduler, torch_dtype=torch.float16,
#     vae=vae, tokenizer=tokenizer
# )
pipeline = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float32, use_safetensors=True, variant="fp32")
pipeline.vae.to(dtype=torch.float32)
pipeline.to(dtype=torch.float32)
# tokenizer = MultiTokenCLIPTokenizer.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", subfolder="tokenizer")
# vae = AutoencoderKL.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", subfolder="vae").to(dtype=torch.float16)
# scheduler = DDIMScheduler.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", subfolder="scheduler")

NameError: name 'torch' is not defined

In [ ]:
# 设置一些路径和参数
# from inversion_free import gen_inversion_free
from utils import img2latent, latent2img
from datetime import datetime
from PIL import Image
import os
from ip_adapter import IPAdapterXL

ip_pipeline = IPAdapterXL(pipeline, image_encoder_path, ip_ckpt, device)

# 加载图片
# 第一级目录
img_dir_src_1 = "./samples/"
img_dir_tar_1 = img_dir_src_1
# 第二级目录
img_dir_src_2 = "pikachu"
src_img_dir = os.path.join(img_dir_src_1, img_dir_src_2)
tar_img_dir_2 = "squirrel"
tar_img_dir = os.path.join(img_dir_tar_1, tar_img_dir_2)


def get_image_file(path):
    img_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')
    img_files = [f for f in os.listdir(path) if f.lower().endswith(img_extensions)]
    return img_files[0]


src_img_file = get_image_file(src_img_dir)
tar_img_file = get_image_file(tar_img_dir)

pipeline = pipeline.to(device)

src_image = Image.open(os.path.join(src_img_dir, src_img_file))
tar_image = Image.open(os.path.join(tar_img_dir, tar_img_file))

src_img_embedding, _ = ip_pipeline.get_image_embeds(src_image)
tar_img_embedding, null_image_embeddings = ip_pipeline.get_image_embeds(tar_image)
src_img_embedding = src_img_embedding.float()
tar_img_embedding = tar_img_embedding.float()
print(src_img_embedding.dtype)
print(src_img_embedding.shape, tar_img_embedding.shape)

src_origin_latent = img2latent(os.path.join(src_img_dir, src_img_file), pipeline)
tar_origin_latent = img2latent(os.path.join(tar_img_dir, tar_img_file), pipeline)
print("src_latent", src_origin_latent.shape)
seed = 12
generator = torch.Generator(device=device)
generator.manual_seed(seed)

torch.float32
torch.Size([1, 4, 2048]) torch.Size([1, 4, 2048])
src_latent torch.Size([1, 4, 64, 64])


In [ ]:
from inversion_free import *

@torch.no_grad()
def diffusion_noise_pred(
        pipeline: DiffusionPipeline,
        noised_latent,
        time_step,
        img_embeddings,
        null_image_embeddings,
        cfg_guidance=None,
):
    positive_text_embeddings, negative_text_embeddings, pooled_text_embeddings, negativate_pooled_text_embeddings = pipeline.encode_prompt(
        "a car with a bright pattern, best quality, high quality",
        num_images_per_prompt=1,
        do_classifier_free_guidance=True,
        negative_prompt="no head, monochrome, lowres, bad anatomy, worst quality, low quality, with figure",
    )
    context_c = torch.concat([positive_text_embeddings, img_embeddings], dim=1).to(pipeline.device)
    context_uc = torch.concat([negative_text_embeddings, null_image_embeddings], dim=1).to(pipeline.device)
    context = torch.concat([context_uc, context_c]).to(pipeline.device)
    noised_latents = torch.concat([noised_latent, noised_latent]).to(pipeline.device)
    print("context", context.shape)
    #prepare added time ids & embeddings 
    add_text_embeds = pooled_text_embeddings
    if pipeline.text_encoder_2 is None:
        text_encoder_projection_dim = int(pooled_text_embeddings.shape[-1])
    else:
        text_encoder_projection_dim = pipeline.text_encoder_2.config.projection_dim
    
    original_size = (512, 512)
    target_size = (512, 512)
    crops_coords_top_left = (0, 0)
    add_time_ids = pipeline._get_add_time_ids(
        original_size,
        crops_coords_top_left,
        target_size,
        dtype=src_img_embedding.dtype,
        text_encoder_projection_dim=text_encoder_projection_dim,
    )
    
    added_cond_kwargs = {"text_embeds": add_text_embeds, "time_ids": add_time_ids.to(device)}
    print("noise_latent", noised_latents.dtype)
    print(context.dtype)
    print("time_step", time_step.dtype)
    print(add_text_embeds.dtype)
    print("time", add_time_ids.dtype)
    noise_pred = pipeline.unet(noised_latents, time_step, encoder_hidden_states=context, added_cond_kwargs=added_cond_kwargs)[0]
    noise_pred_uc, noise_pred_c = noise_pred.chunk(2)
    if cfg_guidance is not None:
        noise_pred = noise_pred_uc + cfg_guidance * (noise_pred_c - noise_pred_uc)
        return noise_pred, noise_pred_uc, noise_pred_c
    else:
        return noise_pred_uc, noise_pred_c

@torch.no_grad()
def gen_inversion_free(
        pipeline: DiffusionPipeline,
        src_origin_latent,
        tar_origin_latent,
        src_img_embeddings,
        tar_img_embeddings,
        num_inference_steps=100,
        inclination="src-pose",
        mode="cosine",
        cfg_guidance=0.7,
        src_coef=0.5,
        direction_coef=0.5,
        strength=0.75,
        generator=None,
        return_all=False,
        null_image_embeddings=None,
        src_work_range=(0, 1),
        backtrace_ratio=0.2,
        backtrace_to=0.4,
        backtrace_loop_num=3,
):
    # generate weights
    src_weights, direction_weights = gen_direction_weights(inclination, mode, num_inference_steps)

    # set the alphas for DDCM
    pipeline.scheduler.set_timesteps(num_inference_steps)

    all_latents = []

    start_step = round(num_inference_steps * (1 - strength))

    callback_to_point = round(num_inference_steps * (1 - backtrace_to))

    src_direction_start_step = round(num_inference_steps * src_work_range[0])
    src_direction_end_step = round(num_inference_steps * src_work_range[1])

    callback_point = round(num_inference_steps * (1 - backtrace_ratio))

    # Iterations
    src_latent_pred_z_0, gen_latent_pred_z_0 = torch.randn_like(src_origin_latent), torch.randn_like(tar_origin_latent)
    init_noise = randn_tensor(
        src_origin_latent.shape,
        device=pipeline.device,
        generator=generator,
        dtype=torch.float32
        )
    # print(pipeline.scheduler)
    src_latent_pred_z_t = inversion_free_add_noise(pipeline, src_origin_latent,
                                                   pipeline.scheduler.timesteps[start_step].int(),
                                                   init_noise)
    gen_latent_pred_z_t = src_latent_pred_z_t.clone().detach()
    i = start_step
    k = backtrace_loop_num
    pbar = tqdm(total=num_inference_steps - start_step)
    global_step = 0
    while start_step <= i < num_inference_steps:
        step = pipeline.scheduler.timesteps[i]
        # step.to(dtype=torch.long)
        # print("new step", step)
        if i == callback_point and k > 0:
            gen_latent_pred_z_t = add_noise(pipeline, gen_latent_pred_z_t,
                                            pipeline.scheduler.timesteps[i].int(),
                                            pipeline.scheduler.timesteps[start_step].int())
            k -= 1
            i = callback_to_point
            pbar.reset()
            continue
        src_weight_t = src_weights[i] * src_coef
        if i < src_direction_start_step or i > src_direction_end_step:
            src_weight_t = 0
        direction_weight = direction_weights[i] * direction_coef
        src_pred_eps, _, _ = diffusion_noise_pred(pipeline, src_latent_pred_z_t, step, src_img_embeddings,
                                                  null_image_embeddings,
                                                  cfg_guidance)
        gen_pred_eps, _, _ = diffusion_noise_pred(pipeline, gen_latent_pred_z_t, step, tar_img_embeddings,
                                                  null_image_embeddings,
                                                  cfg_guidance)
        gen_pred_eps_with_src_prompt, _, _ = diffusion_noise_pred(pipeline, gen_latent_pred_z_t, step,
                                                                  src_img_embeddings,
                                                                  null_image_embeddings,
                                                                  cfg_guidance)

        src_delta_t = get_delta(pipeline, src_origin_latent, src_latent_pred_z_t, step.int())
        src_direction_t = src_delta_t - src_pred_eps

        gen_pred_eps = (gen_pred_eps * (1 - src_weight_t)
                        + src_weight_t * (gen_pred_eps_with_src_prompt + direction_weight * src_direction_t))
        global_step += 1
        print(i)
        gen_pred_output = pipeline.scheduler.step(gen_pred_eps, step, gen_latent_pred_z_t)
        gen_latent_pred_z_t_minus_1= gen_pred_output.prev_sample
        gen_latent_pred_z_t = gen_latent_pred_z_t_minus_1
        if return_all:
            all_latents.append(gen_latent_pred_z_0)
        i += 1
        pbar.update(1)
    if return_all:
        return all_latents
    else:
        return torch.cat([src_latent_pred_z_0.to(pipeline.device), gen_latent_pred_z_t_minus_1])



In [ ]:
from attention_control import make_controller

# 设置输出目录并生成
date = datetime.now().strftime("%Y-%m-%d")
output_dir = f"./output_img/{str(date)}-{img_dir_src_2}-to-{tar_img_dir_2}-IP/"
os.makedirs(output_dir, exist_ok=True)

time = datetime.now().strftime("%H-%M-%S")
num_inference_steps = 30
save_all = False

for src_coefficient in [0.3]:
    for strength in [0.6,]:
        for k in [1]:

            # 选择权重的类型
            inclination = "src-pose"
            mode = "cosine"
            cfg_guidance_scale = 4.5

            # 生成图片
            latents = gen_inversion_free(
                pipeline,
                src_origin_latent, tar_origin_latent,
                src_img_embedding, tar_img_embedding,
                num_inference_steps=num_inference_steps,
                mode=mode, inclination=inclination,
                cfg_guidance=cfg_guidance_scale,
                src_coef=src_coefficient, 
                return_all=save_all,
                generator=generator, 
                null_image_embeddings=null_image_embeddings,
                strength=strength,
                backtrace_ratio=0.2,
                backtrace_to=strength,
                backtrace_loop_num=k,
            )
            # latents = torch.cat([latents[0].unsqueeze(0) * 2], dim=0)
            # print(latents.shape)
            output_img_name = f"{src_img_file[:-4]}2{tar_img_file[:-4]}_{num_inference_steps}steps_{inclination}_{mode[:3]}_src{src_coefficient}_strength{strength}_k{k}"

            if save_all:
                for i, latent in enumerate(latents):
                    img = latent2img(latent[0].detach(), pipeline, generator=generator)
                    print(f"Saving {output_img_name}_{i}.png in {output_dir}")
                    img.save(os.path.join(output_dir, f"{output_img_name}_{i}.png"))
            else:
                img = latent2img(latents[1].detach(), pipeline, generator=generator)
                print(f"Saving {output_img_name}.png in {output_dir}")
                img.save(os.path.join(output_dir, f"{output_img_name}.png"))

context torch.Size([2, 81, 2048])
noise_latent torch.float32
torch.float32
time_step torch.float32
torch.float32
time torch.float32


RuntimeError: expected scalar type Float but found Half

In [ ]:
print(latents[1], latents[0])

tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [na